In [ ]:
import tensorflow as tf
# from tensorflow.keras.applications import InceptionV3
# from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation



In [9]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
batchsize = 8



In [11]:
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=0.2, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2,
                                   height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    r'/Users/kronos27/Desktop/Driver Drowsiness Detection/data/train/',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(
    r'/Users/kronos27/Desktop/Driver Drowsiness Detection/data/train/',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical', subset='validation')



Found 65340 images belonging to 2 classes.
Found 16335 images belonging to 2 classes.


In [12]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_data = test_datagen.flow_from_directory(r'/Users/kronos27/Desktop/Driver Drowsiness Detection/data/test/',
                                             target_size=(80, 80), batch_size=batchsize, class_mode='categorical')



Found 3223 images belonging to 2 classes.


In [13]:
bmodel = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False



94765736/94765736 [==============================] - 39s 0us/step


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint


In [16]:

checkpoint = ModelCheckpoint(r'/Users/kronos27/Desktop/Driver Drowsiness Detection/ProjectData/models_ResNet50/',
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3, )

callbacks = [checkpoint, earlystop, learning_rate]



In [17]:
model.compile(optimizer='Sgd', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_data, steps_per_epoch=train_data.samples // batchsize,
                    validation_data=validation_data,
                    validation_steps=validation_data.samples // batchsize,
                    callbacks=callbacks,
                    epochs=5)



Epoch 1/5


/var/folders/qf/9rcp0yhj5yq2gjz_bxb4cwp00000gn/T/ipykernel_18091/1634758443.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, steps_per_epoch=train_data.samples // batchsize,
2023-04-10 22:06:15.548449: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4005/8167 [=============>................] - ETA: 1:31 - loss: 0.6968 - accuracy: 0.5025

KeyboardInterrupt: 

In [ ]:
loss_tr, acc_tr =  model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
loss_vr, acc_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
loss_test, acc_test = model.evaluate(test_data)
print(loss_test)
print(acc_test)